In [1]:
#pip install Faker

In [4]:
#pip install sqlalchemy

In [5]:
from faker import Faker
import csv
import pandas as pd
from sqlalchemy import create_engine
import os

def generate_records(num_records, services):
    """Generates a list of fake records.""" 
    fake = Faker()
    records = []
    for _ in range(num_records):
        record = {
            'cust_name': fake.name(),
            'address': fake.address(),
            'email': fake.email(),
            'phone_number': fake.phone_number(),
            'transaction_activity': fake.random_element(elements=services),
            'customer_preference': fake.random_element(elements=('app', 'website')),
            'communication_method': fake.random_element(elements=('email', 'phone number'))
        }
        records.append(record)
    return records

def write_records_to_csv(records, file_name):
    """Writes the list of records to a CSV file."""
    with open(file_name, 'w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=records[0].keys())
        writer.writeheader()
        writer.writerows(records)
        print('file generated: ',file_name)

def read_csv_and_insert_to_db(file_path, engine, table_name, company_name):
    """Reads a CSV file into a DataFrame, adds a company name column, and inserts it into a PostgreSQL table."""
    df = pd.read_csv(file_path)
    df['company'] = company_name  # Add a new column with the company name
    df.to_sql(table_name, engine, if_exists='append', index=False)
    print('records inserted for: ',company_name)

def main(folder_path, db_params, table_name, num_records_per_company, services_by_company):
    """Generates records, writes them to CSV, and inserts them into a PostgreSQL database."""
    # Create a database engine
    connection_string = f"postgresql://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['dbname']}"
    engine = create_engine(connection_string)

    # Generate and write records for each company to CSV files
    for company, services in services_by_company.items():
        records = generate_records(num_records_per_company, services)
        file_name = f'{company.lower().replace(" ", "_")}_data.csv'
        write_records_to_csv(records, file_name)

    # Iterate over the created CSV files and insert data into the database
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            # Extract the company name from the file name (excluding the '_data.csv' extension)
            company_name = file_name[:-9]
            read_csv_and_insert_to_db(file_path, engine, table_name, company_name)

if __name__ == '__main__':
    # Database connection parameters
    db_params = {
        'dbname': 'postgres',
        'user': 'postgres',
        'password': 'postgres',
        'host': 'localhost',
        'port': '5432'
    }

    # Path to the folder containing CSV files
    folder_path = '.'

    # Name of the table where you want to insert the data
    table_name = 'company_customers'

    # Define the number of records per company
    num_records_per_company = 100000

    # Define the services for each company
    services_by_company = {
        'GCB': ['Retail banking', 'Corporate banking', 'Investment banking','Asset management', 'Treasury services'],
        'Atlantic Catering and Logistic Ltd': ['Catering', 'Logistic Support'],
        'Federal Commodities Ltd': ['Trading', 'Import/Export', 'Distribution','Brokerage services'],
        'Kek Insurance Brokers Ltd':['Insurance brokerage','risk assessment','policy placement', 'claims management'],
        'VFS Ghana Private Limited':['visa application processing','passport renewal','document authentication','administrative services'],
        'Alpha Tnd Ltd':['Renewal Generation - Solar','Power Transmission','Power Distribution', 'Manufacturing Energy Meters', 'Manufacturing HT & LT Metering Panels'],
        'Fidelity Bank Ghana Ltd':['Retail banking','corporate banking','investment banking','wealth management','treasury services'],
        'Buck Press Ltd':[ 'Printing', 'offset printing', 'security solutions'],
        'Absa':['Retail banking','corporate banking', 'investment banking', 'asset management','treasury services'],
        'Kwahu Praso Rural Bank':['Rural banking services', 'loans', 'microfinance', 'financial advisory']
    }

    # Run the main function
    main(folder_path, db_params, table_name, num_records_per_company, services_by_company)


file generated:  gcb_data.csv
file generated:  atlantic_catering_and_logistic_ltd_data.csv
file generated:  federal_commodities_ltd_data.csv
file generated:  kek_insurance_brokers_ltd_data.csv
file generated:  vfs_ghana_private_limited_data.csv
file generated:  alpha_tnd_ltd_data.csv
file generated:  fidelity_bank_ghana_ltd_data.csv
file generated:  buck_press_ltd_data.csv
file generated:  absa_data.csv
file generated:  kwahu_praso_rural_bank_data.csv
records inserted for:  absa
records inserted for:  alpha_tnd_ltd
records inserted for:  atlantic_catering_and_logistic_ltd
records inserted for:  buck_press_ltd
records inserted for:  federal_commodities_ltd
records inserted for:  fidelity_bank_ghana_ltd
records inserted for:  gcb
records inserted for:  kek_insurance_brokers_ltd
records inserted for:  kwahu_praso_rural_bank
records inserted for:  vfs_ghana_private_limited
